In [1]:
from utils import *
from Bio import SeqIO
from concurrent.futures import ProcessPoolExecutor
from tqdm.auto import tqdm
import pandas as pd

In [32]:
ref_tobamo = parse_fasta('/home/tobamo/analize/project-tobamo/analysis/whole_orf/data/refs_tobamo_nt/tobamo_ref_genomes_57_.fasta')
ref_virga = parse_fasta('/home/tobamo/analize/project-tobamo/analysis/whole_orf/data/refs_tobamo_nt/virgaviridae_RNA1_nt_20_.fasta')
refs = {**ref_tobamo, **ref_virga}

unclassified_tobamo_path = '/home/tobamo/analize/project-tobamo/analysis/whole_orf/data/List_unclasiffied_virga_51_50unique.txt'
with open(unclassified_tobamo_path) as file: unclassified_on_tobamo_tree = [line.strip() for line in file.readlines()]

contigs_palmscan_pos = SeqIO.to_dict(SeqIO.parse('/home/tobamo/analize/project-tobamo/analysis/whole_orf/data/test_contigs_2024-04-09_palmscan_pos_.fasta', 'fasta'))
uncl_contigs_nt_on_tree = {key: contigs_palmscan_pos[key] for key in unclassified_on_tobamo_tree if key in contigs_palmscan_pos}

In [3]:
args_list = []

for s1 in uncl_contigs_nt_on_tree.values():
    for s2 in refs.values():
        args_list.append([s1, s2])

In [4]:
with ProcessPoolExecutor(max_workers=40) as executor:
    results = list(tqdm(executor.map(compute_identity_score_orfxref, args_list), total=len(args_list)))

  0%|          | 0/3850 [00:00<?, ?it/s]

In [8]:
# pd.DataFrame(results).to_csv('results/pairwise_aln_nt/unclass_tobamo_tree_nt_x_ref.csv', index=False)

In [9]:
results = pd.read_csv('results/pairwise_aln_nt/unclass_tobamo_tree_nt_x_ref.csv')

In [33]:
def df_orf_edit(results):
    df = pd.DataFrame(results)
    df = df.rename(columns={'orf_name':'contig_name', 'orf_len':'contig_len', 'aln_orf_len':'aln_contig_len'})
    df["SRR"] = df["contig_name"].str.extract(r"_([A-Za-z0-9]+)$")
    df = df[
        [
            "SRR",
            "contig_name",
            "ref_name",
            "identity_score",
            "gap_openings",
            "gap_ratio",
            "N/aln_len",
            "aln_contig_len",
            "M",
            "N",
            "aln_len",
            "contig_len",
            "ref_len",
        ]
    ]
    return df

In [34]:
df = df_orf_edit(pd.DataFrame(results))

In [35]:
df.to_csv('results/pairwise_aln_nt/unclass_tobamo_tree_nt_x_ref_edited.csv', index=False)